# Energy

> Energy modules for benchmarking

In [ ]:
#| default_exp energy

In [ ]:
#| export
from __future__ import annotations
import math, os, time, warnings
from dataclasses import dataclass, asdict
from typing import Dict, Sequence

import torch

from fasterbench.core import _device_ctx, _sync

try:
    from codecarbon import EmissionsTracker, OfflineEmissionsTracker
except ImportError:
    EmissionsTracker = OfflineEmissionsTracker = None

In [ ]:
#| export
@dataclass
class EnergyMetrics:
    mean_watts : float         # average power during tracked block
    energy_wh  : float         # Wh per *inference*
    co2_eq_g   : float         # g CO₂-eq per inference

    def as_dict(self):
        return asdict(self)


#| export
@torch.inference_mode()
def compute_energy(
    model       : torch.nn.Module,
    x           : torch.Tensor,
    *,
    device      : str | torch.device = "cuda",
    warmup      : int  = 20,
    steps       : int  = 100,
    offline     : bool = True,
    country_iso : str  | None = None,
    measure_secs: int  = 1,
) -> EnergyMetrics:

    if EmissionsTracker is None:   # CodeCarbon missing
        warnings.warn("codecarbon not installed – returning NaNs.")
        return EnergyMetrics(*(math.nan,)*4)

    Tracker = OfflineEmissionsTracker if offline else EmissionsTracker
    tracker = Tracker(
        project_name="fasterbench",
        country_iso_code=(country_iso or os.getenv("NNBENCH_ISO", "USA")),
        measure_power_secs=measure_secs,
        save_to_file=False,
        log_level="error",
    )

    with _device_ctx(device) as dev:
        model = model.eval().to(dev)
        x     = x.to(dev, non_blocking=True)

        # warm-up
        for _ in range(warmup):
            model(x)
        _sync(dev)

        # track
        tracker.start()
        t0 = time.perf_counter()
        for _ in range(steps):
            model(x)
        _sync(dev)
        tracker.stop()
        dur_s = time.perf_counter() - t0

    ene_kwh = tracker.final_emissions_data.energy_consumed  # kWh total
    co2_kg  = tracker.final_emissions                       # kg  total
    mean_w  = (ene_kwh * 3600_000) / dur_s                  # kWh → J (×3.6 MJ), /s → W

    return EnergyMetrics(
        mean_watts = mean_w,
        energy_wh  = (ene_kwh * 1_000) / steps,             # Wh / inf
        co2_eq_g   = (co2_kg  * 1_000) / steps,             # g  / inf
        #duration_s = dur_s / steps,
    )


#| export
def compute_energy_multi(
    model   : torch.nn.Module,
    x       : torch.Tensor,
    *,
    devices : Sequence[str | torch.device] | None = None,
    **kw,
) -> Dict[str, EnergyMetrics]:
    if devices is None:
        devices = ["cpu"]
        if torch.cuda.is_available():
            devices.append("cuda")

    out = {}
    for d in devices:
        try:
            out[str(d)] = compute_energy(model, x, device=d, **kw)
        except RuntimeError:                     # e.g. GPU asked but not present
            out[str(d)] = EnergyMetrics(*(math.nan,)*4)
    return out